# 1. Data loading

In [1]:
from src.data import get_questions_df

questions = get_questions_df()

/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/src/data.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Category"] = datasets[sheet_name]["Category"+suffix]
/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/src/data.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["Category"] = datasets[sheet_name]["Category"+suffix]
/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/src/data.py:28: SettingWithCopyWarning: 
A value is trying to be set on a cop

# 2. LLM challenger Answer a question

In [2]:
question_sample = questions.sample(1)

In [3]:
from typing import TypedDict

class Test(TypedDict):
    x:str

Test(x="a")

{'x': 'a'}

In [4]:
from src.consts import SELECTED_LLMS
from src.answer import llms_answer_questions


/home/lambert-bruyas/miniconda3/envs/gpt_pursuit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
question_answers = llms_answer_questions(question_sample["Question"].tolist(), SELECTED_LLMS)

/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/models/HuggingFaceTB/SmolLM2-360M-Instruct


Device set to use cpu


/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/models/HuggingFaceTB/SmolLM2-135M-Instruct


Device set to use cpu


/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/models/google/flan-t5-small


...

Device set to use cpu


/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/models/google/gemma-2-2b-it


Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  8.78it/s]
Device set to use cpu


/home/lambert-bruyas/external_git/gpt-pursuit/data-acquisition/models/Qwen/Qwen2.5-1.5B-Instruct


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  5.32it/s]
Device set to use cpu


In [ ]:
for i, llm in enumerate(SELECTED_LLMS):
    model_answers:list[str] = []
    for j, question in enumerate(question_answers[i]):
        answer = question_answers[i][j][0]['generated_text']
        if isinstance(answer, list):
            answer = answer[-1]['content']
        else:
            pass
        model_answers.append(answer)
        print(answer, llm.pipeline_kwargs["model"])
    question_sample.loc[:, llm.pipeline_kwargs["model"]] = model_answers

The recording engineer on "Sgt. Pepper's Lonely Hearts Club Band" was John Lennon. HuggingFaceTB/SmolLM2-360M-Instruct
Apples, pears, and other fruits were commonly stored in pub cellars during the 17th century. HuggingFaceTB/SmolLM2-360M-Instruct
The answer depends on the specific insect species being referred to. However, if we're talking about Hymenoptera, which includes ants, bees, wasps, and butterflies, the insect with the prominent pair of pincers at the tip of its abdomen is the female praying mantis. Pheromone spray is also a characteristic pheromone used by male praying mantises to attract females. HuggingFaceTB/SmolLM2-360M-Instruct
South America is closer to the Antarctic. HuggingFaceTB/SmolLM2-360M-Instruct
A leporine cat is a feline cat that belongs to the family of small, typically nocturnal mammals known as the LEPOROINS. The most common examples of leporine cats include the Leopard Longhair, Persian, and Cornish Rex. HuggingFaceTB/SmolLM2-360M-Instruct
Harry Potter is 

/tmp/ipykernel_9886/1240836090.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  question_sample.loc[:, llm.pipeline_kwargs["model"]] = model_answers
/tmp/ipykernel_9886/1240836090.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  question_sample.loc[:, llm.pipeline_kwargs["model"]] = model_answers
/tmp/ipykernel_9886/1240836090.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [ ]:
import difflib
from typing import TypedDict, NotRequired
import pandas as pd
from types import Question
from datetime import datetime

question_output:list[Question] = []

for i in range(len(question_sample)):
    question_output.append({
        "id": int(question_sample.iloc[i]["id"]),
        "category": question_sample.iloc[i]["Category"],
        "question": question_sample.iloc[i]["Question"],
        "expected_answer": question_sample.iloc[i]["Answer"],
        "llm_answers": {llm.pipeline_kwargs["model"]: {
            "answer": question_sample.iloc[i].loc[llm.pipeline_kwargs["model"]], 
            "is_correct": difflib.SequenceMatcher(None, question_sample.iloc[i]["Answer"], question_sample.iloc[i].loc[llm.pipeline_kwargs["model"]]).ratio() > 0.8} for llm in SELECTED_LLMS},
        "difficulty": question_sample.iloc[i]["Difficulty"] if not pd.isna(question_sample.iloc[i]["Difficulty"]) else None
    })
import json

with open("question_output.json", "w") as f:
    json.dump(question_output, f, indent=4)


In [ ]:
question_answers

[[[{'generated_text': [{'role': 'user',
      'content': 'You are participating in a trivia contest. Answer the following question: Who was the recording engineer on The Beatles Album "Sgt. Pepper\'s Lonely Hearts Club Band"?'},
     {'role': 'assistant',
      'content': 'The recording engineer on "Sgt. Pepper\'s Lonely Hearts Club Band" was John Lennon.'}]}],
  [{'generated_text': [{'role': 'user',
      'content': 'You are participating in a trivia contest. Answer the following question: Apart from drinks what used to be stored in pub cellars'},
     {'role': 'assistant',
      'content': 'Apples, pears, and other fruits were commonly stored in pub cellars during the 17th century.'}]}],
  [{'generated_text': [{'role': 'user',
      'content': "You are participating in a trivia contest. Answer the following question: Which Insect Has A Pair Of Prominent Pincers At The Tip Of It's Abdomen\xa0"},
     {'role': 'assistant',
      'content': "The answer depends on the specific insect spe

In [ ]:
[(answer[0]['generated_text'], llm.pipeline_kwargs["model"]) for llm_answer, llm in zip(question_answers, SELECTED_LLMS) for answer in llm_answer]

[([{'role': 'user',
    'content': 'You are participating in a trivia contest. Answer the following question: Who was the recording engineer on The Beatles Album "Sgt. Pepper\'s Lonely Hearts Club Band"?'},
   {'role': 'assistant',
    'content': 'The recording engineer on "Sgt. Pepper\'s Lonely Hearts Club Band" was John Lennon.'}],
  'HuggingFaceTB/SmolLM2-360M-Instruct'),
 ([{'role': 'user',
    'content': 'You are participating in a trivia contest. Answer the following question: Apart from drinks what used to be stored in pub cellars'},
   {'role': 'assistant',
    'content': 'Apples, pears, and other fruits were commonly stored in pub cellars during the 17th century.'}],
  'HuggingFaceTB/SmolLM2-360M-Instruct'),
 ([{'role': 'user',
    'content': "You are participating in a trivia contest. Answer the following question: Which Insect Has A Pair Of Prominent Pincers At The Tip Of It's Abdomen\xa0"},
   {'role': 'assistant',
    'content': "The answer depends on the specific insect s